In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import json

# Question 1

## Part 1
### Preprocessing

In this section we will go through the data 
Reading the data and some preprations
<br>

The training contains float data but they are in string format so we must split them
delimiter is space and the first character is space, So we will go through string data and convert it to float 


In [7]:
#################### read dataset ####################
def process_string_dataset(string_data, delimiter = ' '):
    """
    convert the data we have in string to float
    string_data can be lines of strings
    """
    ## count of data lines
    data_line_count = len(string_data)

    ## Data is our data values in float
    data_floats = []
    ## create a temp string to save each numbers character
    temp_string = ''

    ## go through the data and convert it to float
    for index in range(0, data_line_count):
        data_floats.append([])
        for character in string_data[index][1:]:
            if(character != delimiter and character != '\n'):
                temp_string += character
            else:
                ## if the string goes to delimiter convert the data into float
                ## and reset temp_string variable to save the next data
                data_floats[index].append(float(temp_string))
                temp_string = ''
                
    return data_floats



def read_dataset(name):
    """
    read our given dataset and return the datas
    name is the dataset directory with its name can be forexample: ../toy/Atrain
    """
    file = open(name, 'r')

    lines = file.readlines()
    data_lines = []
    for line in lines:
        ## dataset with # in the start does not contain data
        if(line[0] != '#'):
            data_lines.append(line)
            
    ## convert to float and return the ready dataset
    float_data = process_string_dataset(data_lines)
    return float_data

In [8]:
train_raw_data = read_dataset('toy dataset/Atrain')
np.array(train_raw_data).shape

(4, 1000)

since the data was 2*1000 (For each class) we create two matrixes <br>
two matrixes was made and the two shows the 2 classes we have


In [9]:
## create the claases matrix as in file
train_class1 = np.matrix(train_raw_data[:2])
train_class2 = np.matrix(train_raw_data[2:])

Convert Dataset into pandas dataframe 

In [10]:
def attach_classes(data_frames, labels):
    """
    Consider each dataframe a class of our data
    attach the dataframes into one and add the class lebels
    Inputs:
    data_frames is an array of data_frames
    lebels is the labels to the classes
    """
    
    data_frame_count = len(data_frames)
    
    ## create an empty dataframe
    df = pd.DataFrame()
    
    for i in range(0, data_frame_count):
        ## create the label array for each class with the class data count
        class_label = np.full(len(data_frames[i]), labels[i])
        ## add the label to each class
        data_frames[i]['label'] = class_label 
        df = df.append(data_frames[i], ignore_index=True)
        
    return df

## convert each class to a dataframe
def convert_to_pd_dataframe(matrix_data):
    """
    convert a dataset in the form of matrix into pandas dataframe
    Inputs:
    matrix_data with two feature space
    """
    df = pd.DataFrame(matrix_data, columns=['feature_one', 'feature_two']) 
    
    return df
def convert_rowData_df(row_Datas, labels):
    """
    convert raw data into dataframes
    Input:
    row_Datas must be an array with the raw datas,
    if we had one raw data then it must be as [raw_data]
    each raw_data is for one class
    
    lebels is the labels to the data classes
    """
    ## create an array containing the dataframes 
    df_class_arrays = []
    
    ## get the count of row datas needed to be converted
    count = len(row_Datas)
    
    ## go through each row data array and create get a dataframe for each
    for i in range(0, count):
        df = convert_to_pd_dataframe(np.matrix(row_Datas[i]))
        df_class_arrays.append(df)
    
    ## attach the dataframes of each row data (each row data represent a class)
    df = attach_classes(df_class_arrays, labels)
    
    return df
        

In [11]:
df_train = convert_rowData_df([train_class1.T, train_class2.T], [0, 1])
df_train.head()

,feature_one,feature_two,label
0,-0.813596,1.423864,0
1,-0.347811,-0.494301,0
2,0.188346,-0.277319,0
3,0.167306,0.179168,0
4,-0.899921,1.323836,0


### implementing KNN algorithm
As we know about the algorithm it uses the k nearest neigber to check which class it belongs 

In [12]:
## Set the k value
K = 5

In [13]:
## load test dataset
test_raw_data = read_dataset('toy dataset/Atest')
## get classes and convert to matrix representation
test_raw_class1 = np.array(test_raw_data).T[1000:]
test_raw_class1 = np.matrix(test_raw_class1)

test_raw_class2 = np.array(test_raw_data).T[:1000]
test_raw_class2 = np.matrix(test_raw_class2)

df_test = convert_rowData_df([test_raw_class1, test_raw_class2], [0, 1])

To calculate the Euclidean distance we use the formula below
$$
\begin{equation}
    d = \sqrt{(x_1 - x_2)^2 + (y_1 - y_2)^2}
\end{equation}
$$

And our dataset has just two feature and then we can expand the above formula to use as below ( For each class we have to calculate the formula) 
$$
\begin{equation}
d = \sqrt{(feature\_one_{test} - feature\_one_{class1})^2 + (feature\_two_{test} - feature\_two_{class1})^2}
\end{equation}
$$
$$
\begin{equation}
d = \sqrt{(feature\_one_{test} - feature\_one_{class2})^2 + (feature\_two_{test} - feature\_two_{class2})^2}
\end{equation}
$$
*Note:* the equations needed to be calculated for each point. <br>
*Note 2:* The Pythagorean theorem is the more generalized of the equation in above. 

In [27]:
## this is same as above

## this variable is made to save the result of distances
distances = []

## start the test phase
for i in range(0, 2000):
    d = (df_train['feature_one'] - df_test.iloc[i]['feature_one']) ** 2
    d += (df_train['feature_two']- df_test.iloc[i]['feature_two']) ** 2
    
    d = np.sqrt(d)
    distances.append(d)

In [15]:
test_label_votes = []
for i in range(0,2000):
    ## get the nearest K points
    d = distances[i].sort_values()[:K]
    nearest_indexes = d.index
    
    labels = df_train.iloc[nearest_indexes].label
    
    ## use the mean to calculate the average vote
    vote = labels.mean()
    
    test_label_votes.append(vote)


In [16]:
## the threshold that uses to classify test data
THRESHOLD = 0.5
test_labels = np.array(test_label_votes) > THRESHOLD
test_labels = [1 if label else 0 for label in test_labels]
# classify = (test_labels == df_test.label).values

In [18]:
## check the classifier labels with actual labels
equal_labels = (test_labels == df_test.label)
equal_labels.sum()

107

we can see that 107 of test data are correctly classified with $K = 5$ <br>
Now we will capsulate the calculations in a function

In [19]:
def reform_data(data ,THRESHOLD):
    """
    Reform an array of floats between one and zero with the threshold value into integers as 0 and 1
    Inputs:
    THRESHOLD is the value to set the data whether into 0 or 1 
    data is the float array
    """
    test_labels = np.array(data) >= THRESHOLD
    
    return test_labels

############################# check labels of the classified data #############################
def check_label(actual_labels, target_labels):
    """
    check the labels of the classifier output with actual data
    
    Inputs:
    actual_labels are the labels that the classifier gave us
    target_labels are the correct labels of the data
    """
    
    ## convert to numpy arrays to compute the equality of each elemnts
    a_labels = np.array(actual_labels)
    t_labels = np.array(target_labels)
    
    ## mark as True the eqaul labels
    equal_labels = (a_labels == t_labels)
    
    return equal_labels
    

def KNN(K, train_data, test_data, features, threshold = 0.5):
    """
    The K nearest neighbor algorithm
    
    Inputs:
    K is the KNN algorithm parameter (a positive integer value)
    train_data and test_data in the format of pandas dataframe, the algorithm's food!
    features are the feature for each data
    threshold is the value to decide the result of KNN algorithm
    """
        
    ## get the algorithm start time
    start_time = time.time_ns()
    
    assert K > 0, 'K must be a positive value and also not zero!'

    test_label_votes = []
    
    ## start the test phase
    for index in range(0, len(test_data)):
        
        d = 0
        for feature in features:
            d += (train_data[feature]- test_data.iloc[index][feature]) ** 2
        d = np.sqrt(d)

        ## the index of K nearest neighbor points
        nearest_indexes = d.sort_values()[:K].index
                
        ## get the labels of nearest data
        labels = train_data.iloc[nearest_indexes].label
        
        ## use the mean to calculate the average vote
        vote = labels.mean()
        test_label_votes.append(vote)
        
    ## decide the voting process in KNN
    classified_data = reform_data(test_label_votes, threshold)

    finish_time = time.time_ns()
    print('Finished! algorithm time: ', ((finish_time - start_time) / 1000), ' miliseconds')
        
    return classified_data


In [20]:
features = np.array(df_test.columns[df_test.columns != 'label'])

check the memory and time usage of the algorithm

In [21]:
%memit test_labels = KNN(5 , df_train, df_test, features)

Finished! algorithm time:  2109561.173  miliseconds
peak memory: 174.25 MiB, increment: 0.48 MiB


As you can see the increment was about 0 mega Bytes, This means that it does not really consume alot of memory

In [22]:
result = check_label(test_labels, df_test.label)
result.sum()

107

#### Now its time to check different K values

In [23]:
## remove the target feature
features = np.array(df_test.columns[df_test.columns != 'label'])

## save the result of different K values
labels_classified = {}
for K in [1, 2, 5, 10, 15, 20, 50, 100, 500, 1000]:
    label = KNN(K, df_train, df_test, features)
    labels_classified[str(K)] = label

Finished! algorithm time:  2172647.918  miliseconds
Finished! algorithm time:  2058391.065  miliseconds
Finished! algorithm time:  2026945.805  miliseconds
Finished! algorithm time:  2090292.86  miliseconds
Finished! algorithm time:  2039694.83  miliseconds
Finished! algorithm time:  2038825.182  miliseconds
Finished! algorithm time:  2042348.485  miliseconds
Finished! algorithm time:  2065345.877  miliseconds
Finished! algorithm time:  2127289.311  miliseconds
Finished! algorithm time:  2073999.929  miliseconds


In [24]:
for key, item in labels_classified.items():
    ## check correct classification results
    result = check_label(item, df_test.label)
    print('K=%s ' % key, 'True classified %s' % result.sum(), '\taccuracy: %f' % (result.sum()*100 / len(result)))


K=1  True classified 153 	accuracy: 7.650000
K=2  True classified 186 	accuracy: 9.300000
K=5  True classified 107 	accuracy: 5.350000
K=10  True classified 91 	accuracy: 4.550000
K=15  True classified 90 	accuracy: 4.500000
K=20  True classified 91 	accuracy: 4.550000
K=50  True classified 92 	accuracy: 4.600000
K=100  True classified 94 	accuracy: 4.700000
K=500  True classified 104 	accuracy: 5.200000
K=1000  True classified 123 	accuracy: 6.150000


## Part 2

Use the *Itrain* dataset <br>
split it for into 100 for validation set and 900 for training  <br>
*NOTE: we have processed the data into csv format using convert_to_csv.py script and saved them into processed_dataset directory*

In [ ]:
! python3 convert_to_csv.py

In [69]:
Itrain_df = pd.read_csv('processed_dataset/Itrain.csv')

features = Itrain_df.columns[Itrain_df.columns != 'label']

history = {}
for K in [1, 2, 3, 5, 7, 9, 10]:
    ## use 900 of each class for training    
    df_training = Itrain_df[Itrain_df.label == 0][:900]
    df_training = df_training.append(Itrain_df[Itrain_df.label == 1][:900])
    
    ## now we have a dataframe with 1800 values but the indexes are not set properly
    ## Since we are directly working with index in our algorithm, we need to reset it
    df_training = df_training.reset_index(drop=True)

    ## use 100 of each class for validation
    df_validation = Itrain_df[Itrain_df.label == 0][900:]
    df_validation = df_validation.append(Itrain_df[Itrain_df.label == 1][900:])
    
    ## now we have a dataframe with 200 values but the indexes are not set properly
    ## Since we are directly working with index in our algorithm, we need to reset it
    df_validation = df_validation.reset_index(drop=True)
    
    label = KNN(K, df_training, df_validation, features)
    
    ## check correct classification results
    result = check_label(label, df_validation.label)
    
    print('K=%s ' % K, 'True classified count %s from %s' % (result.sum(), len(result)), '\taccuracy: %f' % (result.sum()*100 / len(result)))    
    
    
    

Finished! algorithm time:  414542.223  miliseconds
K=1  True classified count 200 from 200 	accuracy: 100.000000
Finished! algorithm time:  404241.222  miliseconds
K=2  True classified count 200 from 200 	accuracy: 100.000000
Finished! algorithm time:  403420.411  miliseconds
K=3  True classified count 200 from 200 	accuracy: 100.000000
Finished! algorithm time:  402239.077  miliseconds
K=5  True classified count 200 from 200 	accuracy: 100.000000
Finished! algorithm time:  403473.398  miliseconds
K=7  True classified count 200 from 200 	accuracy: 100.000000
Finished! algorithm time:  403851.638  miliseconds
K=9  True classified count 200 from 200 	accuracy: 100.000000
Finished! algorithm time:  401086.326  miliseconds
K=10  True classified count 200 from 200 	accuracy: 100.000000


## Appendix A
run the knn.py script just to save the outputs in the notebook 

In [26]:
## run the knn.py script and save the history to json file
! python3 knn.py

Preforming A Dataset (ATrain.csv, ATest.csv)
-----------------------------------------------
Finished! K=1 algorithm time:  10912130.759  miliseconds
K=1  correctly classified count: 153 	accuracy: 7.650000
Finished! K=2 algorithm time:  10947507.039  miliseconds
K=2  correctly classified count: 186 	accuracy: 9.300000
Finished! K=5 algorithm time:  11171008.564  miliseconds
K=5  correctly classified count: 107 	accuracy: 5.350000
Finished! K=10 algorithm time:  11103951.541  miliseconds
K=10  correctly classified count: 91 	accuracy: 4.550000
Finished! K=15 algorithm time:  11593564.561  miliseconds
K=15  correctly classified count: 90 	accuracy: 4.500000
Finished! K=20 algorithm time:  11487735.531  miliseconds
K=20  correctly classified count: 91 	accuracy: 4.550000
Finished! K=50 algorithm time:  10913763.468  miliseconds
K=50  correctly classified count: 92 	accuracy: 4.600000
Finished! K=100 algorithm time:  10976115.961  miliseconds
K=100  correctly classified count: 94 	accurac

## Appendix B
create the confusion matrix of the results using the KNN-Result.json

In [2]:
dict_data = ''
with open('KNN-Result.json', 'r') as json_result:
    dict_data = json.load(json_result)
json_result.close()

In [19]:
############################# READ DATASET #############################
def read_csv_datasets(dataset_prefixes):
    """
    read the datasets in csv format and prepare them for KNN algorithm
    
    INPUT:
    -----------
    dataset_prefixes:  An array of prefixes, example: ['A','B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
    
    OUTPUTS:
    -----------
    train_ds_array:  the array of multiple train datasets
    test_ds_array:  the array of multiple test datasets
    """
    ## save the datasets into an array
    train_ds_array = []
    test_ds_array = []
    
    ## read them and append it to arrays
    for char in dataset_prefixes:
        df_train = pd.read_csv('processed_dataset/%ctrain.csv' % char)
        df_test = pd.read_csv('processed_dataset/%ctest.csv' % char)
        
        train_ds_array.append(df_train)
        test_ds_array.append(df_test)
    
    return train_ds_array, test_ds_array

def create_confusion_matrix(actual_labels, target_labels):
    """
    check the labels of the classifier output with actual data
    
    INPUTS:
    -----------
    actual_labels:  are the labels that the classifier gave us
    target_labels:  are the correct labels of the data
    
    OUTPUT:
    -----------
    confusion_matrix:  A Dataset with with elements of TP, TN, FP, FN as True Positives, True Negatives, False Positives and False Negative
    """
    
    a = np.array(actual_labels)
    t = np.array(target_labels)
    
    TP = ((a == True) & (t == True)).sum()
    TN = ((a == False) & (t == False)).sum()
    FP = ((a == True) & (t == False)).sum()
    FN = ((a == False) & (t == True)).sum()
 
    confusion_matrix = pd.DataFrame(data={'TP':[TP], 'TN': [TN], 'FP': [FP], 'FN': [FN]})
    
    return confusion_matrix

### Now is the time to compute confusion matrix for each values

In [72]:
K_values = [1, 2, 5, 10, 15, 20, 50, 100, 500, 1000]
prefixes = ['A','B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']


ds_confusion_matrix = pd.DataFrame(columns=[ 'dataset', 'K' ,'TP', 'TN', 'FP', 'FN'])

## just use the Atrain, Atest results
for dataset_prefix in prefixes:

    ## get the dataset name for json file
    dataset_name = 'dataset %c' % dataset_prefix
    
    ## get the test dataframe to check the confusion matrix
    _ , test_ds = read_csv_datasets([dataset_prefix])
    
    
    ## the K value in KNN algorithm
    for K in K_values:
        classified_values = []
        for j in range(0, len(dict_data[dataset_name][str(K)])):
            values = dict_data[dataset_name][str(K)][ 'index_%i_classified' % j ]
            classified_values.append(values)
            
        
        ds = create_confusion_matrix(classified_values, test_ds[0].label)
        ds['K'] = K
        ds['dataset'] = dataset_prefix
            
        ## add the confusion matrix values into the dataframe
        ds_confusion_matrix = ds_confusion_matrix.append(ds, ignore_index=True)
        

In [74]:
ds_confusion_matrix.head(12)

,dataset,K,TP,TN,FP,FN
0,A,1,77,76,924,923
1,A,2,135,51,949,865
2,A,5,34,73,927,966
3,A,10,25,66,934,975
4,A,15,18,72,928,982
5,A,20,22,69,931,978
6,A,50,19,73,927,981
7,A,100,17,77,923,983
8,A,500,7,97,903,993
9,A,1000,4,119,881,996
